<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [48]:
import requests
import json
import folium
import ipywidgets as widgets
from IPython.display import display, Markdown
%reload_ext version_information
%version_information folium, json, IPython, ipywidgets, requests

Software versions
Python 3.7.4 64bit [Clang 10.0.1 (clang-1001.0.46.4)]
IPython 7.7.0
OS Darwin 18.7.0 x86_64 i386 64bit
folium 0.10.0
json 2.0.9
IPython 7.7.0
ipywidgets 7.5.1
requests 2.22.0
Thu Sep 12 13:57:09 2019 CEST

In [58]:
def app():
    wEnv=widgets.Dropdown(
        options={'Production': 'https://api.resourcewatch.org', 'Staging': 'https://staging-api.globalforestwatch.org', 'Local':'http://0.0.0.0:9000'},
        value='https://api.resourcewatch.org',
        description='Env: ',
    )

    wIso=widgets.Dropdown(
        description='Country: ',
        options={'COG': 'COG', 'COD': 'COD', 'CAF':'CAF'},
        value='COG',
    )

    #mapArea = widgets.Output()
    
    def updateMap(env, iso):
        url = f'{env}/v1/geostore/admin/{iso}'
        r = requests.get(url)
        if r.status_code == 200:

            m = folium.Map()
            folium.GeoJson(
                r.json()['data']['attributes']["geojson"], name='geostore'
            ).add_to(m)

            folium.LayerControl().add_to(m)
            bbox = r.json()['data']['attributes']["bbox"]
            m.fit_bounds([[bbox[1],bbox[0]],[bbox[3],bbox[2]]])
            display(m)
            display(Markdown(f"""```json
            {json.dumps(r.json()['data']['attributes']['info'], sort_keys=True, indent=2)}``` """))

        else:
            raise r.status_code

        

    out = widgets.interactive_output(updateMap, {'env':wEnv,'iso':wIso})
    header = widgets.VBox([wEnv,wIso])
    app = widgets.AppLayout(header=header,
          left_sidebar=header,
          center=out,
          footer=None)
    return app
app()

AppLayout(children=(VBox(children=(Dropdown(description='Env: ', options={'Production': 'https://api.resourcew…